In [ ]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

from config import *
import scipy.io
relevent_columns = dep_var1 + dep_var2 + ['刀盘功率']

In [ ]:
# I'm so fucked writing this
test1_fns = sorted(Path('tbmData/test/附件1：必选课题一测试数据集').glob('*.mat'))
column_names = pd.read_csv('tbmData/data/CREC188_20160809.txt', delimiter='\t', nrows=1).columns # get column names from training data

In [ ]:
test1_fns

# Save test data as feathers

Assert the last column of test data is all zero

In [ ]:
def read_test_mat_as_df(mat_fn, columns):
    test_input = scipy.io.loadmat(mat_fn)['a']
    test_df = pd.DataFrame(test_input[:,:-1], columns=columns[2:])
    return test_df

save_dir = Path('tbmData/test/test1_feathers')
for mat_fn in test1_fns:
    df = read_test_mat_as_df(mat_fn, column_names)
    save_feather_fn = save_dir/(mat_fn.stem + '.feather')
    df.to_feather(str(save_feather_fn))

context = Context('save-after-removing-redundent-columns', fn_cycles='tbmData/test/test1_feathers', debug=0, mulr=1, load_data=False)
fmtr = DataFormatter(context, cycle_feathers=context.fn_cycles)
cycles = fmtr.cycles

In [ ]:
cycle_dir = Path('plots/test_segments')
cycle_dir.mkdir(exist_ok=True)
def plot_cycle(i,full=False):
    cycle_str = 'cycle_' + str(i)
    axs = plots(cycles.loc[i,relevent_columns].iloc[:None if full else 400], cols=5, title=cycle_str, unit_figsize=(7, 3)).flatten()
#     for ax in axs: 
#         ax.axvline(context.idx[i])
#         ax.axvline(context.idx[i]+30)
    fig = plt.gcf()
    fig.savefig(str(cycle_dir/cycle_str), dpi=96)
    plt.close(fig)

In [ ]:
with ProcessPoolExecutor() as e:
    e.map(plot_cycle, cycles.index.levels[0])

In [ ]:
list(map(plot_cycle, cycles.index.levels[0]))

determine the correctness of segmentation by checking mean and std

In [ ]:
c_test = cycles.loc[:,relevent_columns]

c_test_starts = [c_test.loc[i].iloc[0] for i in cycles.index.levels[0]]

c_test_starts = pd.DataFrame(c_test_starts).reset_index(drop=True)

In [ ]:
c_test = cycles.loc[:,relevent_columns]

c_test_ends = [c_test.loc[i].iloc[-1] for i in cycles.index.levels[0]]

c_test_ends = pd.DataFrame(c_test_ends).reset_index(drop=True)

# ours

In [ ]:
# without noise
context = Context('removed_redundent1-full-30-no_noise', debug=0, mulr=0, load_data=False)
fmtr = DataFormatter(context, cycle_feathers=context.fn_cycles)

In [ ]:
fmtr.cycles.

In [ ]:
cycles = fmtr.cycles
idx = fmtr.beginning_index(thresh=700)

c_train = cycles.loc[:,relevent_columns]
c_train_starts = [c_train.loc[i].iloc[j] for i, j in zip(cycles.index.levels[0], idx)]
c_train_starts = pd.DataFrame(c_train_starts).reset_index(drop=True)

In [ ]:
display_all(c_test_starts.mean(0))
display_all(c_train_starts.mean(0))

In [ ]:
display_all(c_train_starts.mean(0))

In [ ]:
display_all(c_test_starts.std(0))
display_all(c_train_starts.std(0))

In [ ]:
display_all(c_train_starts.std(0))

In [ ]:
for i in range(4):
    plt.figure(figsize=(20,5))
    ax = sns.distplot(c_train_starts.iloc[:,i])
    set_ax_font(ax)

In [ ]:
for i in range(4):
    plt.figure(figsize=(20,5))
    ax = sns.distplot(c_test_starts.iloc[:,i])
    set_ax_font(ax)

In [ ]:
display_all(c_test_ends.mean(0))
display_all(c_train_ends.mean(0))

In [ ]:
display_all(c_test_ends.std(0))
display_all(c_train_ends.std(0))

In [ ]:
c_train = cycles.loc[:,relevent_columns]

c_train_ends = [c_train.loc[i].iloc[-1] for i in cycles.index.levels[0]]

c_train_ends = pd.DataFrame(c_train_ends).reset_index(drop=True)